## Problem 1: LRU Cache

A quick recap on the solution requirements and what this translates into for usage of data structures. To achieve get & put operations in constant time (O(1)), I'm going to use a hashmap. In addition, removing the oldest item when the cache is full is required in constant time as well, thus I cannot use a simple array/list, as e.g. removing at the head and inserting in the tail, would lead to index updating which is not a constant operation. I'm going to implement this part as a queue.

In [1]:
# Your work here
# Creating a node class for the queue
class Node:
    def __init__(self, key, value):
        # My Node class receives and holds inputs (key and value) as well as pointers for the next and previous nodes, respectively
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

In [17]:
# Creating a LRU Cache data structure
class LRU_Cache(object):

    def __init__(self, capacity):
        # Initialize class variables, starting with a hashmap using Python's dict object as well as variable for current list lengt
        self.hashmap = {}
        self.num_entries = 0
        # Including a head and tail parameter here instead own class
        self.head = None
        self.tail = None
        # Place an upper bound on the size of the cache
        self.cap = capacity

    def get(self, key):
        """ 
        Options to handle:
        - Cache hit: head or not; update head if not
        - Cache miss
        """
        # Retrieve item from provided key. Return -1 if nonexistent. 
        # Cache hit
        if key in self.hashmap:
            # Case 1: it's the head
            node = self.hashmap[key]
            if node == self.head:
                return node.value
            # Case 2: not head, needs to become new head
            else:
                # Delete node for current position (!= head)
                self.delete(node)
                # Set selected node as new head
                self.set_head(node)
                return node.value      
        
        # Cache miss
        if key not in self.hashmap:
            return -1

    def put(self, key, value):
        """
        Options to handle:
        - Check Capacity
        
        """
        # If cache is at capacity remove the oldest item.
        if self.num_entries == 5:
            self.delete_tail()
            self.num_entries -= 1
        elif self.cap <= 0:
            raise ValueError("Capacity is too low. It must be above 0!")
        
        new_node = Node(key,value)
        
        # If cache is empty, start new one as linked list
        if self.num_entries == 0:
            self.head = new_node
            self.tail = new_node
            self.hashmap[key] = new_node
            self.num_entries += 1
            return
        
        # Set the value if key is not present in the cache.
        if key not in self.hashmap:
            self.set_head(new_node)
            self.hashmap[key] = new_node
        
        # Update the value if key is present in the cache.
        if key in self.hashmap:
            current_node = self.hashmap[key]
            current_node.value = value
        # Check if current node is already head node, update if not
            if self.head != current_node:
                self.delete(current_node)
                self.set_head(current_node)
     
    def set_head(self, node):
        # Set current node to head node
        # Update pointers
        # Check if there is a head already
        if not self.head:
            self.head = node
            self.tail = node
        # If self.head is already defined, update pointer and set new head
        else:
            node.prev = self.head
            self.head.next = node
            self.head = node
            
        self.num_entries += 1
           
    def delete(self, node):
        # Delete node and update pointers
        # Check if there is a head at all
        if not self.head:
            return
        
        # If self.head is already set, but there are no pointer to prev and next (case with one node that is head & tail)
        # Reset head/tail
        if not node.next and not node.prev:
            self.head = None
            self.tail = None
        
        # Case for node being a middle node
        if node.next:
            node.next.prev = node.prev
        if node.prev:
            node.prev.next = node.next
            
        # Case for node being the tail node
        if self.tail == node:
            self.delete_tail()
        
        self.num_entries -= 1
        
    def delete_tail(self):
        # Delete key in hashmap first
        del self.hashmap[self.tail.key]
        # Delete/ Update tail and pointer
        node = self.tail.next
        node.prev = None
        self.tail = node
    

In [19]:
# Testing
our_cache = LRU_Cache(5)

our_cache.put(1, 1);
our_cache.put(2, 2);
our_cache.put(3, 3);
our_cache.put(4, 4);


print(our_cache.get(1))       # returns 1
print(our_cache.get(2))       # returns 2
print(our_cache.get(9))      # returns -1 because 9 is not present in the cache

our_cache.put(5, 5) 
our_cache.put(6, 6)

our_cache.get(3)      # returns -1 because the cache reached it's capacity and 3 was the least recently used entry


1
2
-1


-1

Time complexity comments - Goal: All operations must take O(1) time.
Due to the implementation using a hasmap/dict object as well as a linked list, all operations fullfil requirement of constant time complexity O(1).

## Problem 2: File Recursion - Finding Files

For this problem, the goal is to write code for finding all files under a directory (and all directories beneath it) that end with ".c"

Here is an example of a test directory listing, which can be downloaded:

- ./testdir
- ./testdir/subdir1
- ./testdir/subdir1/a.c
- ./testdir/subdir1/a.h
- ./testdir/subdir2
- ./testdir/subdir2/.gitkeep
- ./testdir/subdir3
- ./testdir/subdir3/subsubdir1
- ./testdir/subdir3/subsubdir1/b.c
- ./testdir/subdir3/subsubdir1/b.h
- ./testdir/subdir4
- ./testdir/subdir4/.gitkeep
- ./testdir/subdir5
- ./testdir/subdir5/a.c
- ./testdir/subdir5/a.h
- ./testdir/t1.c
- ./testdir/t1.h

Python's os module will be useful—in particular, you may want to use the following resources:

os.path.isdir(path)

os.path.isfile(path)

os.listdir(directory)

os.path.join(...)

In [23]:
import os

def find_files(suffix, path):
    """
    Find all files beneath path with file name suffix.

    Note that a path may contain further subdirectories
    and those subdirectories may also contain further subdirectories.

    There are no limit to the depth of the subdirectories can be.

    Args:
      suffix(str): suffix if the file name to be found
      path(str): path of the file system

    Returns:
       a list of paths
    """
    # Create empty list to collect all correct paths
    path_list = []
    
    # Create list of all elements in current path
    dir_list = os.listdir(path)
    
    for element in dir_list:
        # create full path of element
        sub_path = path + "/" + element
        # Option 1: Check if directory or not
        if os.path.isdir(sub_path):
            # recurse through sub directory if check is True
            sub_list = find_files(suffix, sub_path)
            # Check if function returns a file path for matching file, None if empty list
            if sub_list != None:
                path_list.extend(sub_list)
        # Option 2: check if element is a file and if so, check suffix match
        elif os.path.isfile(sub_path):
            if element[-2:] == suffix:
                path = os.path.join(path,element)
                path_list.append(path)
            else:
                continue
    
    return path_list


In [24]:
#Testing
find_files(".c","./testdir")

['./testdir/subdir5/a.c',
 './testdir/subdir3/subsubdir1/b.c',
 './testdir/subdir1/a.c',
 './testdir/t1.c']

In [7]:
## Locally save and call this file ex.py ##

# Code to demonstrate the use of some of the OS modules in python

import os

# Let us print the files in the directory in which you are running this script
print (os.listdir("."))

# Let us check if this file is indeed a file!
print (os.path.isfile("./ex.py"))

# Does the file end with .py?
print ("./ex.py".endswith(".py"))

['testdir.zip', '.ipynb_checkpoints', 'testdir', 'Workbook.ipynb', 'ex.py']
True
True


## Problem 3: Huffman Coding

A Huffman code is a type of optimal prefix code that is used for compressing data. The Huffman encoding and decoding schema is also lossless, meaning that when compressing the data to make it smaller, there is no loss of information.

The Huffman algorithm works by assigning codes that correspond to the relative frequency of each character for each character. The Huffman code can be of any length and does not require a prefix; therefore, this binary code can be visualized on a binary tree with each encoded character being stored on leafs.

There are many types of pseudocode for this algorithm. At the basic core, it is comprised of building a Huffman tree, encoding the data, and, lastly, decoding the data.

Here is one type of pseudocode for this coding schema:

- Take a string and determine the relevant frequencies of the characters.
- Build and sort a list of tuples from lowest to highest frequencies.
- Build the Huffman Tree by assigning a binary code to each letter, using shorter codes for the more frequent letters. (This is the heart of the Huffman algorithm.)
- Trim the Huffman Tree (remove the frequencies from the previously built tree).
- Encode the text into its compressed form.
- Decode the text from its compressed form.


You then will need to create encoding, decoding, and sizing schemas. You have to uniquely account for each ASCII code, i.e. you need to take care of -

- Individual character of the string

- All punctuation marks

- Spaces

- Lowercase and Uppercase

Hint: You can solve the problem with mere `TreeNode` objects that have left and right pointers to other `TreeNode` objects. Having access to the root node let's you traverse the entire tree.

Huffman Visualization!(https://people.ok.ubc.ca/ylucet/DS/Huffman.html)

Tree Generator(http://huffman.ooz.ie/)

Additional Explanation (https://www.siggraph.org/education/materials/HyperGraph/video/mpeg/mpegfaq/huffman_tutorial.html)

In [6]:
import sys

def huffman_encoding(data):
    pass

def huffman_decoding(data,tree):
    pass

if __name__ == "__main__":
    codes = {}

    a_great_sentence = "The bird is the word"

    print ("The size of the data is: {}\n".format(sys.getsizeof(a_great_sentence)))
    print ("The content of the data is: {}\n".format(a_great_sentence))

    encoded_data, tree = huffman_encoding(a_great_sentence)

    print ("The size of the encoded data is: {}\n".format(sys.getsizeof(int(encoded_data, base=2))))
    print ("The content of the encoded data is: {}\n".format(encoded_data))

    decoded_data = huffman_decoding(encoded_data, tree)

    print ("The size of the decoded data is: {}\n".format(sys.getsizeof(decoded_data)))
    print ("The content of the encoded data is: {}\n".format(decoded_data))

## Problem 4: Active Directory

In Windows Active Directory, a group can consist of user(s) and group(s) themselves. We can construct this hierarchy as such. Where User is represented by str representing their ids.

In [ ]:
class Group(object):
    def __init__(self, _name):
        self.name = _name
        self.groups = []
        self.users = []

    def add_group(self, group):
        self.groups.append(group)

    def add_user(self, user):
        self.users.append(user)

    def get_groups(self):
        return self.groups

    def get_users(self):
        return self.users

    def get_name(self):
        return self.name


parent = Group("parent")
child = Group("child")
sub_child = Group("subchild")

sub_child_user = "sub_child_user"
sub_child.add_user(sub_child_user)

child.add_group(sub_child)
parent.add_group(child)

Write a function that provides an efficient look up of whether the user is in a group.

In [ ]:
def is_user_in_group(user, group):
    """
    Return True if user is in the group, False otherwise.

    Args:
      user(str): user name/id
      group(class:Group): group to check user membership against
    """
    return None

## Problem 5: Blockchain

A Blockchain is a sequential chain of records, similar to a linked list. Each block contains some information and how it is connected related to the other blocks in the chain. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data. For our blockchain we will be using a SHA-256 hash, the Greenwich Mean Time when the block was created, and text strings as the data.

Use your knowledge of linked lists and hashing to create a blockchain implementation.

<Blockchain Picture
    
 We can break the blockchain down into three main parts. First is the information hash:

In [ ]:
import hashlib

def calc_hash(self):
      sha = hashlib.sha256()

      hash_str = "We are going to encode this string of data!".encode('utf-8')

      sha.update(hash_str)

      return sha.hexdigest()

We do this for the information we want to store in the block chain such as transaction time, data, and information like the previous chain.

The next main component is the block on the blockchain:

In [ ]:
class Block:

    def __init__(self, timestamp, data, previous_hash):
      self.timestamp = timestamp
      self.data = data
      self.previous_hash = previous_hash
      self.hash = self.calc_hash()

Above is an example of attributes you could find in a Block class.

Finally you need to link all of this together in a block chain, which you will be doing by implementing it in a linked list. All of this will help you build up to a simple but full blockchain implementation!